# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [1]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

In [2]:
import gymnasium as gym
import gym_hearthstone

env = gym.make('Hearthstone-v1')

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 26268 cards
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>

IMPLEMENTED CARDS: 7963
UNIMPLEMENTED CARDS: 18305
Implemented but Uncollectible: 6059
CardClass.DEATHKNIGHT amount of cards is: 0
CardClass.DRUID amount of cards is: 130
CardClass.HUNTER amount of cards is: 133
CardClass.MAGE amount of cards is: 128
CardClass.PALADIN amount of cards is: 128
CardClass.PRIEST amount of cards is: 119
CardClass.ROGUE amount of cards is: 123
CardClass.SHAMAN amount of cards is: 126
CardClass.WARLOCK amount of cards is: 118
CardClass.WARRIOR amount of cards is: 120
CardClass.DEMONHUNTER amount of cards is: 41
CardClass.NEUTRAL amount of cards is: 738
HearthstoneEnv - Version 0.2.0
>>> Initialize PLAYER1
>>> Initialize PLAYER2
>>> Initialize GAME
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [ ]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN

In [ ]:
model = DQN("MultiInputPolicy", env, verbose=1)

In [ ]:
import time
import os
print(os.getpid())


steps = 100
env.reset_stats()
start = time.time()
env.reset()

model.learn(total_timesteps=steps, log_interval=4)

end = time.time()
env.display_stats()
print("Avg steps per game: " + str(steps / env.curr_episode))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(end-start, (end-start) / env.curr_episode) )


env.close()

## Time
# 100 steps - 9.8s
# 1000 steps - 95.5s

## Memory
# 1000 steps - 725 Kb

## Win Rate & Reward 
# 1000 steps - 0.43 & -3
#

In [3]:
env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    print(action)
    observation, reward, terminated, info = env.step(action)

    if terminated:
        print("################################## New Game ############################################")
        observation = env.reset()

env.close()
print("Game finished !!!!!!!!!!!!!!!!!!!")

C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:190: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:203: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady 

>>> Initialize PLAYER1
>>> Initialize PLAYER2
>>> Initialize GAME
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
105
2449
2960
2964
4791
2824
3814
5680
1832
2
------------------------
>>> Current Step 1
------------------------

>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
doing single action for AI[<Move.choice: 5>, 1]


[fireplace.card]: <Spell ('Crushing Walls')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.card]: <Spell ('Wing Blast')> moves from <Zone.HAND: 3> to <Zone.DECK: 2>
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.entity]: Player1 begins turn 2
[fireplace.entity]: Player1 is now at 2 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.entity]: Player1 draws <Spell ('Secret Plan')>
[fireplace.card]: <Spell ('Secret Plan')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:235: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of t

818
4868
105
2449
2964
4791
2824
3814
5680
1832
0
------------------------
>>> Current Step 2
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 4, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> possible_actions 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
doing single action for AI[<Move.play_card: 4>, 1, None, None]
818
4868
105
2964
2449
4791
2824
3814
5680
1832
0
------------------------
>>> Current Step 3
------------------------
{'c

[fireplace.actions]: Player1 plays <Minion ('Spellshifter')> (target=None, index=None)
[fireplace.entity]: Player1 pays 2 mana
[fireplace.card]: <Minion ('Spellshifter')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Spellshifter')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Spellshifter')>]
[fireplace.actions]: Activating <Minion ('Spellshifter')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: <Minion ('Loot Hoarder')> attacks <Hero ('Garrosh Hellscream')>
[fireplace.actions]: <Minion ('Loot Hoarder')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=2)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Loot Hoarder')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=2)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Loot Hoa

...

{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1], [<Move.minion_attack: 2>, 1, 0], [<Move.minion_attack: 2>, 1, 1]], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}
[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.hero_power: 1>, None, None], [<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1], [<Move.minion_attack: 2>, 1, 0], [<Move.minion_attack: 2>, 1, 1], [<Move.end_turn: 0>]]
doing single turn for rando
Doing action: [<Move.play_card: 4>, 3, None, None]
{'choice': [], 'play_card': [[<Move.play_card: 4>, 2, None, Non

[fireplace.actions]: <Hero ('Garrosh Hellscream')> attacks <Minion ('Jeweled Macaw')>
[fireplace.actions]: <Hero ('Garrosh Hellscream')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=4)> targeting [<Minion ('Jeweled Macaw')>]
[fireplace.actions]: <Hero ('Garrosh Hellscream')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=4)> targeting [<Minion ('Jeweled Macaw')>]
[fireplace.actions]: <Hero ('Garrosh Hellscream')> triggering <TargetedAction: Damage()> targeting [<Minion ('Jeweled Macaw')>]
[fireplace.actions]: <Minion ('Jeweled Macaw')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=1)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Jeweled Macaw')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=1)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Jeweled Macaw')> triggering <TargetedAction: Damage()> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: <Hero ('Garrosh Hellscream')> loses 1 armor instead of dam

>>> possible_actions 17:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None], [<Move.hero_power: 1>, None, None], [<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1], [<Move.minion_attack: 2>, 0, 2], [<Move.minion_attack: 2>, 0, 3], [<Move.hero_attack: 3>, None, 0], [<Move.hero_attack: 3>, None, 1], [<Move.hero_attack: 3>, None, 2], [<Move.hero_attack: 3>, None, 3], [<Move.end_turn: 0>]]
doing single action for AI[<Move.hero_attack: 3>, None, 1]
3822
2977
1833
7783
1935
2933
3579
2787
1337
3170
2357
1966
98
0
------------------------
>>> Current Step 21
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.p

[fireplace.actions]: <Minion ('Nerubian')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=4)> targeting [<Hero ('Rexxar')>]
[fireplace.actions]: <Minion ('Nerubian')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=4)> targeting [<Hero ('Rexxar')>]
[fireplace.actions]: <Minion ('Nerubian')> triggering <TargetedAction: Damage()> targeting [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player2 plays <Minion ('Arcanosmith')> (target=None, index=None)
[fireplace.entity]: Player2 pays 4 mana
[fireplace.card]: <Minion ('Arcanosmith')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Arcanosmith')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Arcanosmith')>]
[fireplace.actions]: Activating <Minion ('Arcanosmith')> action targeting None
[fireplace.actions]: <Minion ('Arcanosmith')> triggering <TargetedAction: Summon(<Summon.CARD>='KAR_710m')> targeting [Pla

3822
2977
1833
7783
1935
2933
3579
2787
1337
3170
2357
1966
98
3
------------------------
>>> Current Step 23
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None]], 'heropower': [], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> possible_actions 6:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 6:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None], [<Move.end_turn: 0>]]
doing single action for AI[<Move.play_card: 4>,